In [1]:
# from bioimageio_chatbot.utils import ChatbotExtension
from schema_agents import schema_tool
from schema_agents.tools.code_interpreter import CodeInterpreter
from pydantic import Field, BaseModel
from typing import Optional, List, Dict, Any
import pandas as pd
from pathlib import Path
import requests
import re
import os
import asyncio
from imjoy_rpc.hypha import connect_to_server, login

class MalariaExplorer:
    def __init__(self, work_dir_root: str):
        self.data_path = "complete_patient_data.csv"
        self.work_dir_root = work_dir_root
        work_dir_root = "./.data"
        os.makedirs(work_dir_root, exist_ok=True)
        self.ex = CodeInterpreter(work_dir_root=work_dir_root)

    def malaria_data_explorer(self,
                                   query: str = Field(..., description="Query to explore the malaria data"),
                                   ):
        """Based on the query, this tool will explore the malaria data by writing some python code to be applied on the csv data. """
        malaria_data = pd.read_csv(self.data_path)
        query_to_ex = f"Answer user's question {query} using the malaria data: {malaria_data}"
        result = self.ex.execute_query(query_to_ex)
        print(result)
        return result

2024-04-17 14:48:55.905 | INFO     | schema_agents.config:__init__:44 - Config loading done.


NotConfiguredException: Set OPENAI_API_KEY first

In [ ]:
malaria_exp = MalariaExplorer(work_dir_root="./.data")
chatbot_extension = {
    "id": "malaria",
    "type": "bioimageio-chatbot-extension",
    "name": "Malaria Data Explorer",
    "description": "This tool will explore the malaria data by writing some python code to be applied on the csv data.",
    "tools": {
        "my_tool": malaria_exp.malaria_data_explorer
    }
}



server_url = "https://chat.bioimage.io"
token = await login({"server_url": server_url})
server = await connect_to_server({"server_url": server_url, "token": token})
svc = await server.register_service(chatbot_extension)
print(f"Extension service registered with id: {svc.id}, you can visit the service at: https://bioimage.io/chat?server={server_url}&extension={svc.id}")
